In [4]:
import numpy as np 
import pandas as pd
import os 
import sys 
import torch 
import torchaudio 
import matplotlib.pyplot as plt 
#%matplotlib_inline

import IPython.display as ipd

import torchaudio_functions as Audio
from torchaudio_functions import *

# Dataset Exploritory Analysis 

The RAVDESS (Ryerson Audio-Visual Database of Emotional Speech and Song) is a widely used dataset for emotion classification using recorded speech because of its high quality and consistent audio quality. The dataset can be found at https://smartlaboratory.org/ravdess/ and more info can be found in this offical citation from the creators: 

Livingstone SR, Russo FA (2018) The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS): A dynamic, multimodal set of facial and vocal expressions in North American English. PLoS ONE 13(5): e0196391. https://doi.org/10.1371/journal.pone.0196391.

RAVDESS contains both audio and video but for this project i will only be using and discussions the audio-only portion of the dataset. The database contains audio from 24 actors (12 male, 12 female) each speaking 2 similar sentences in a neutral North American accent. Each statement is spoken in 8 different emotions/ expressions (calm, happy, sad, angry, fearful, suprise, and disgust). Each one is performed in 2 different levels of emotional intensity (normal, strong) and a neutral expression is added. All audio recordings have a sample rate of 48kHz with a bit depth of 16bit. There is a total of 1440 audio files (24 actors X 60 trials per actor).


RAVDESS does not come with any sort of metadata table with information on the recordings but instead the filename themselves have all the information. Each filename has a 7 part numerical identifier (ex. 03-01-04-01-01-02-12.wav). The identifiers represent the following: 

    1.) Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
    2.) Vocal channel (01 = speech, 02 = song).
    3.) Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
    4.) Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the ‘neutral’ emotion.
    5.) Statement (01 = “Kids are talking by the door”, 02 = “Dogs are sitting by the door”).
    6.) Repetition (01 = 1st repetition, 02 = 2nd repetition).
    7.) Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

So for example the file 03-01-04-01-01-02-12.wav contains the following metadata: 

    1.) Audio-only (03)
    2.) Speech (01)
    3.) Sad (04)
    4.) Normal Intensity (01)
    5.) "Kids are talking by the door" (01)
    6.) Second Repetition (02)
    7.) Actor-12 Male (12) 

# Creating Metadata table

To make the audio data easier to deal with I will create a Pandas data frame that will contain the file path of each audio file and linked to it's emotion as it will be are target variable. The audio files are each separated into their own folders by which actor performed them. 

In [10]:
audio_path = '/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/'

dir_list = os.listdir(audio_path)
dir_list.sort()


# creating CSV with labels and path name 
# https://www.kaggle.com/shivamburnwal/speech-emotion-recognition


Ravdess = '/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/'

ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()




,Emotions,Path
0,angry,/Users/stephen/Emotion_Dectection/data/RAVDESS...
1,fear,/Users/stephen/Emotion_Dectection/data/RAVDESS...
2,fear,/Users/stephen/Emotion_Dectection/data/RAVDESS...
3,angry,/Users/stephen/Emotion_Dectection/data/RAVDESS...
4,disgust,/Users/stephen/Emotion_Dectection/data/RAVDESS...


In [11]:
# Lets take a look at our target variables 
Ravdess_df['Emotions'].value_counts()

angry       192
fear        192
disgust     192
sad         192
surprise    192
happy       192
calm        192
neutral      96
Name: Emotions, dtype: int64

In [16]:
Ravdess_df['Path'][0]

'/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/Actor_16/03-01-05-01-02-01-16.wav'

In [12]:
# export as csv 
Ravdess_df.to_csv('/Users/stephen/Emotion_Dectection/data/RAVDESS/Ravdess.csv')

The Dataset is all balanced except for the "neutral" emotion. This doesn't seem like it will be a problem so we'll leave it as is for now. 

# Visualizing the audio data
Here we'll use the Pytorch module torchaudio to convert the .wav files to tensor object so we can visualize the signal and then perform the necessary feature extractions to obtain data to train our model. 

In [21]:
# declare a variable that will locate a wav file to extract from 
angry_file_row = Ravdess_df.loc[Ravdess_df['Emotions'] == 'angry']
test_file_row = angry_file_row.iloc[0]
test_file_row

Emotions                            angry
Path          01/03-01-05-01-02-01-01.wav
Intensity                          normal
Statement    Dogs are sitting by the door
actor                            Actor_01
Name: 2, dtype: object

In [22]:
# based on Pytorch Documents 
# https://pytorch.org/tutorials/beginner/audio_feature_extractions_tutorial.html



# global variables 
n_fft = 2048
n_mels = 128
n_mfcc = 128
win_length = None 
hop_length = 512
sample_rate = None


# load audio file path to tenso 
waveform, sample_rate = torchaudio.load(test_file_row['Path'])

print_stats(waveform, sample_rate=sample_rate)
plot_waveform(waveform, sample_rate=sample_rate)
plot_specgram(waveform, sample_rate)
play_audio(waveform, sample_rate)

formats: can't open input file `01/03-01-05-01-02-01-01.wav': No such file or directory


RuntimeError: Error loading audio file: failed to open file 01/03-01-05-01-02-01-01.wav

In [ ]:
spectrogram = T.Spectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
)
# Perform transformation
spec = spectrogram(waveform)

print_stats(spec)
plot_spectrogram(spec[0], title='torchaudio')

In [ ]:
mel_spectrogram = T.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm='slaney',
    onesided=True,
    n_mels=n_mels,
    mel_scale="htk",
)

melspec = mel_spectrogram(waveform)
plot_spectrogram(
    melspec[0], title="MelSpectrogram - torchaudio", ylabel='mel freq')

In [ ]:
mfcc_transform = torchaudio.transforms.MFCC(
    sample_rate=sample_rate,
    n_mfcc=n_mfcc,
    melkwargs={
      'n_fft': n_fft,
      'n_mels': n_mels,
      'hop_length': hop_length,
      'mel_scale': 'htk',
    })

mfcc = mfcc_transform(waveform)

plot_spectrogram(mfcc[0])

In [ ]:
pitch = torchaudio.functional.detect_pitch_frequency(waveform, sample_rate)
plot_pitch(waveform, sample_rate, pitch)
play_audio(waveform, sample_rate)

In [ ]:
pitch_feature = torchaudio.functional.compute_kaldi_pitch(waveform, sample_rate)
pitch, nfcc = pitch_feature[..., 0], pitch_feature[..., 1]

plot_kaldi_pitch(waveform, sample_rate, pitch, nfcc)
play_audio(waveform, sample_rate)

# Audio Feature Augmentation 
https://pytorch.org/tutorials/beginner/audio_feature_augmentation_tutorial.html

In [ ]:
import torchaudio.functional as F
import torchaudio.transforms as T

# creating a Pytorch dataframe odject


In [ ]:
from torch.utils.data import Dataset
# https://www.youtube.com/watch?v=88FFnqt5MNI&ab_channel=ValerioVelardo-TheSoundofAI

In [7]:
# creating CSV with labels and path name 
# https://www.kaggle.com/shivamburnwal/speech-emotion-recognition


Ravdess = '/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/'

ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

,Emotions,Path
0,angry,/Users/stephen/Emotion_Dectection/data/RAVDESS...
1,fear,/Users/stephen/Emotion_Dectection/data/RAVDESS...
2,fear,/Users/stephen/Emotion_Dectection/data/RAVDESS...
3,angry,/Users/stephen/Emotion_Dectection/data/RAVDESS...
4,disgust,/Users/stephen/Emotion_Dectection/data/RAVDESS...


In [9]:
Ravdess_df['Emotions'].value_counts()

angry       192
fear        192
disgust     192
sad         192
surprise    192
happy       192
calm        192
neutral      96
Name: Emotions, dtype: int64